# **ConductTest Runs**

In [1]:
import pandas as pd
import boto3
import json
import time
import configparser
import psycopg2

In [2]:
# Load DWH Params from a file

config = configparser.ConfigParser()
config.read_file(open('maitys_aws_dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")
DWH_ENDPOINT           = config.get("DWH","DWH_ENDPOINT")
DWH_ROLE_ARN           = config.get("DWH","DWH_ROLE_ARN")
LOG_DATA               = config.get("S3","LOG_DATA")
LOG_JSONPATH           = config.get("S3","LOG_JSONPATH")
SONG_DATA             = config.get("S3","SONG_DATA")
AWS_REGION             = config.get("S3","AWS_REGION")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

In [3]:
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=KEY,
                       aws_secret_access_key=SECRET
                   )

## **Check flat file structure**

In [4]:
# Song Data
bucket_name = "udacity-dend"
sampleDbBucket =  s3.Bucket(bucket_name)
obj_list = list(sampleDbBucket.objects.filter(Prefix='song_data'))
obj = obj_list[1]
file_path = "s3://" + bucket_name + "/" + obj.key
df_song_data = pd.read_json(file_path,  lines=True)
df_song_data.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARJNIUY12298900C91,NaN,,NaN,Adelitas Way,213.9424,1,SOBLFFE12AF72AA5BA,Scream,2009


In [5]:
# Log Data
bucket_name = "udacity-dend"
sampleDbBucket =  s3.Bucket(bucket_name)
obj_list = list(sampleDbBucket.objects.filter(Prefix='log_data'))
obj = obj_list[5]
file_path = "s3://" + bucket_name + "/" + obj.key
df_log_data = pd.read_json(file_path,  lines=True)
df_log_data.head(n=1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,A Fine Frenzy,Logged In,Anabelle,F,0,Simpson,267.91138,free,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",PUT,NextSong,1.541044e+12,256,Almost Lover (Album Version),200,1541377992796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",69


## **Load Data from Raw files to Staging Tables**

In [6]:
# Drop tables if they exist
staging_events_table_drop = "DROP TABLE IF EXISTS public.staging_events;"
staging_songs_table_drop = "DROP TABLE IF EXISTS public.staging_songs;"

staging_events_table_create= ("""
CREATE TABLE IF NOT EXISTS public.staging_events (
    artist TEXT, 
    auth TEXT, 
    firstName TEXT, 
    gender TEXT, 
    ItemInSession TEXT,
    lastName TEXT, 
    length FLOAT8, 
    level TEXT, 
    location TEXT, 
    method TEXT,
    page TEXT, 
    registration TEXT, 
    sessionId TEXT, 
    song TEXT, 
    status TEXT,
    ts BIGINT, 
    userAgent TEXT, 
    userId TEXT);
""")

# Create staging tables
staging_songs_table_create = ("""
CREATE TABLE IF NOT EXISTS public.staging_songs (
    artist_id TEXT,
    artist_latitude FLOAT8,
    artist_longitude FLOAT8, 
    artist_location TEXT, 
    artist_name TEXT,
    duration FLOAT8, 
    num_songs INT, 
    song_id TEXT PRIMARY KEY, 
    title TEXT, 
    year INT);
""")

# Copy data from S3 to staging tables
staging_events_copy = ("""
copy public.staging_events from '{}'
credentials 'aws_iam_role={}'
compupdate off region '{}'
timeformat as 'epochmillisecs'
truncatecolumns blanksasnull emptyasnull
json '{}';                                             
""").format(LOG_DATA, DWH_ROLE_ARN, AWS_REGION, LOG_JSONPATH)

staging_songs_copy = ("""
copy public.staging_songs from '{}'
credentials 'aws_iam_role={}'
format as json 'auto' compupdate off region '{}';
""").format(SONG_DATA, DWH_ROLE_ARN, AWS_REGION)

In [7]:
start_time = time.time()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop tables if they exist
cursor.execute(staging_events_table_drop)
conn.commit()
print("Dropped staging_events table if it exists")

cursor.execute(staging_songs_table_drop)
conn.commit()
print("Dropped staging_songs table if it exists")

# Create staging tables
cursor.execute(staging_events_table_create)
conn.commit()
print("Created staging_events table")

cursor.execute(staging_songs_table_create)
conn.commit()
print("Created staging_songs table")

# Insert data from S3 to staging tables
cursor.execute(staging_events_copy)
conn.commit()
print("Inserted data from S3 to staging_events table")

cursor.execute(staging_songs_copy)
conn.commit()
print("Inserted data from S3 to staging_songs table")

print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Dropped staging_events table if it exists
Dropped staging_songs table if it exists
Created staging_events table
Created staging_songs table
Inserted data from S3 to staging_events table
Inserted data from S3 to staging_songs table
Total time to run: 180.867 seconds


## **Load Sample Data from Staging Tables to Final Tables**

### Songplays table

In [8]:
start_time = time.time()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop table if it exists
songplay_table_drop = "DROP TABLE IF EXISTS public.songplays;"

# Create Final table
songplay_table_create = ("""
CREATE TABLE public.songplays(
    songplay_id BIGINT IDENTITY(0,1) PRIMARY KEY,
    start_time TIMESTAMP NOT NULL,
    user_id TEXT NOT NULL DISTKEY,
    level TEXT,
    song_id TEXT NOT NULL,
    artist_id TEXT NOT NULL,
    session_id TEXT,
    location TEXT,
    user_agent TEXT);
""")

# Copy data from staging table
songplay_table_insert = ("""
INSERT INTO public.songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
SELECT  TIMESTAMP 'epoch' + a.ts/1000 * INTERVAL '1 second' AS start_time, 
        a.userId as user_id,
        a.level,
        b.song_id,
        b.artist_id,
        a.sessionId as session_id,
        a.location,
        a.userAgent as user_agent
FROM    staging_events AS a 
JOIN    staging_songs AS b
ON      a.artist = b.artist_name
AND     a.length = b.duration
AND     a.song = b.title
WHERE   a.page = 'NextSong';
""")

# Execute queries
cursor.execute(songplay_table_drop)
conn.commit()
print("Dropped songplays table if it exists")

cursor.execute(songplay_table_create)
conn.commit()
print("Created songplays table")

cursor.execute(songplay_table_insert)
conn.commit()
print("Inserted data from staging tables to songplays table")

# Print some data from table
query = ("""select * from public.songplays limit 10;""")
cursor.execute(query)
conn.commit()

print("Printing first 10 rows of songplays table")
i = 0
for row in cursor:
    i = i + 1
    print(row)
    if i == 10:
        break
    
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Dropped songplays table if it exists
Created songplays table
Inserted data from staging tables to songplays table
Printing first 10 rows of songplays table
(101, datetime.datetime(2018, 11, 13, 16, 45, 3), '97', 'paid', 'SOHZDYL12A8C141C73', 'AR32RXI1187B9BA168', '537', 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
(189, datetime.datetime(2018, 11, 7, 0, 43), '97', 'paid', 'SOHTKMO12AB01843B0', 'AR5EYTL1187B98EDA0', '293', 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
(48, datetime.datetime(2018, 11, 21, 1, 5, 29), '97', 'paid', 'SORTFPF12A81C2171A', 'ARY5UO61187FB5271F', '671', 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
(112, datetime.datetime(2018, 11, 15, 12, 28, 39), '30', 'paid', 'SOIOESO12A6D4F621D', 'ARVL

### Users table

In [9]:
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop table if it exists
user_table_drop = "DROP TABLE IF EXISTS public.users;"

user_table_create = ("""
CREATE TABLE public.users(
    user_id TEXT PRIMARY KEY,
    first_name TEXT,
    last_name TEXT,
    gender TEXT,
    level TEXT)
    DISTSTYLE AUTO;                    
""")

# Copy data from staging table
user_table_insert = ("""
INSERT INTO public.users (user_id, first_name, last_name, gender, level)
SELECT DISTINCT 
            userId as user_id,
            firstName as first_name,
            lastName as last_name,
            gender,
            level
FROM        public.staging_events
WHERE       page = 'NextSong'
AND         userId IS NOT NULL;
""")

# Execute queries
cursor.execute(user_table_drop)
conn.commit()
print("Dropped users table if it exists")

cursor.execute(user_table_create)
conn.commit()
print("Created users table")

cursor.execute(user_table_insert)
conn.commit()
print("Inserted data from staging tables to users table")

# Print some data from table
query = ("""select * from public.users limit 10;""")
cursor.execute(query)
conn.commit()

print("Printing first 10 rows of users table")
i = 0
for row in cursor:
    i = i + 1
    print(row)
    if i == 10:
        break
    
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Total time to run: 2.8964 seconds
Dropped users table if it exists
Created users table
Inserted data from staging tables to users table
Printing first 10 rows of users table
('89', 'Kynnedi', 'Sanchez', 'F', 'free')
('75', 'Joseph', 'Gutierrez', 'M', 'free')
('42', 'Harper', 'Barrett', 'M', 'paid')
('55', 'Martin', 'Johnson', 'M', 'free')
('10', 'Sylvie', 'Cruz', 'F', 'free')
('62', 'Connar', 'Moreno', 'M', 'free')
('6', 'Cecilia', 'Owens', 'F', 'free')
('54', 'Kaleb', 'Cook', 'M', 'free')
('4', 'Alivia', 'Terrell', 'F', 'free')
('68', 'Jordan', 'Rodriguez', 'F', 'free')
Total time to run: 5.9589 seconds


### Songs Table

In [10]:
start_time = time.time()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop table if it exists
song_table_drop = "DROP TABLE IF EXISTS public.songs;"

# Create table
song_table_create = ("""
CREATE TABLE public.songs(
    song_id TEXT PRIMARY KEY,
    title TEXT,
    artist_id TEXT NOT NULL DISTKEY,
    year INTEGER,
    duration FLOAT8);
""")

# copy data from staging table
song_table_insert = ("""
INSERT INTO public.songs(song_id, title, artist_id, year, duration)
SELECT DISTINCT
    song_id,
    title,
    artist_id,
    year,    
    duration
FROM public.staging_songs;                     
""")

# Execute queries
cursor.execute(song_table_drop)
conn.commit()
print("Dropped songs table if it exists")

cursor.execute(song_table_create)
conn.commit()
print("Created songs table")

cursor.execute(song_table_insert)
print("Inserted data from staging tables to songs table")

# Print some data from table
query = ("""select * from public.songs limit 10;""")
cursor.execute(query)
conn.commit()

print("Printing first 10 rows of songs table")
i = 0
for row in cursor:
    i = i + 1
    print(row)
    if i == 10:
        break
    
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Dropped songs table if it exists
Created songs table
Inserted data from staging tables to songs table
Printing first 10 rows of songs table
('SOXLWPS12AF729D5DA', "Don't Fall", 'ARG6A591187B9B757A', 1983, 232.28036)
('SOIXCFZ12A8AE462F5', 'Tetrispack', 'ARA6BH81187B98F3AF', 2005, 256.31302)
('SOANUHW12A8C137587', "Time You'll Never Get Back", 'ARAFRFP1187FB4230B', 2003, 354.06322)
('SOWTIOB12A8C139D36', 'Without You', 'ARARO7M1187FB5C0B3', 2003, 426.21342)
('SOMLFXG12A8C133076', 'Dark places', 'ARCM19K1187FB3F447', 2008, 266.34404)
('SOTQIGY12AB01892F3', 'Mato por Tu Amor', 'ARUQQ0X1187B9A08E0', 2004, 216.5024)
('SOYJFQC12A6D4F7EE5', 'Universal Garden', 'ARY3ZB61187FB53200', 1993, 470.72608)
('SOESQFB12A8C133EEC', 'We Cry As One', 'ARIGHFP1187B9A4467', 2003, 398.65424)
('SOPGETA12A8C13FDE5', 'Gente Come Noi', 'ARSW7R41187FB38381', 1995, 248.05832)
('SOBVJSH12AC468DE83', 'El Diablo', 'ARO82XH1187B98E18E', 1991, 328.61995)
Total time to run: 2.5479 seconds


### Artists Table

In [11]:
start_time = time.time()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop table if it exists
artist_table_drop = "DROP TABLE IF EXISTS public.artists;"

# Create table
artist_table_create = ("""
CREATE TABLE public.artists(
    artist_id TEXT PRIMARY KEY,
    name TEXT NOT NULL,
    location TEXT,
    lattitude FLOAT8,
    longitude FLOAT8)
    DISTSTYLE AUTO;
""")

# copy data from staging table
artist_table_insert = ("""
INSERT INTO public.artists(artist_id, name, location, lattitude, longitude)
SELECT DISTINCT
    artist_id,
    artist_name,
    artist_location,
    artist_latitude,    
    artist_longitude
FROM public.staging_songs;
""")

# Execute queries
cursor.execute(artist_table_drop)
conn.commit()
print("Dropped artists table if it exists")

cursor.execute(artist_table_create)
conn.commit()
print("Created artists table")

cursor.execute(artist_table_insert)
print("Inserted data from staging tables to artists table")

# Print some data from table
query = ("""select * from public.artists limit 10;""")
cursor.execute(query)
conn.commit()

print("Printing first 10 rows of artists table")
i = 0
for row in cursor:
    i = i + 1
    print(row)
    if i == 10:
        break
    
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Dropped artists table if it exists
Created artists table
Inserted data from staging tables to artists table
Printing first 10 rows of artists table
('ARNE4341187B99F236', 'Jackie Leven', '', None, None)
('ARNJLGR1242078249B', 'Max Avery Lichtenstein', '', None, None)
('ARBFQAE1187FB3E957', 'Stephanie Mills', 'Brooklyn, NY', 40.65507, -73.94888)
('ARH09RT1187FB4A843', 'OV7', '', None, None)
('ARU1MVY1187FB37B73', 'Brainstorm', 'Jelgava, Latvia', None, None)
('ARBUHDB1187FB3E72C', 'Bobby Vee', 'Fargo, ND', 46.87591, -96.78176)
('ARTDQRC1187FB4EFD4', 'Black Eyed Peas / Les Nubians / Mos Def', 'Los Angeles, CA', None, None)
('AR4RBOB1187B9AC209', 'Anastacia', '', None, None)
('ARHUC691187B9AD27F', 'The Mars Volta', 'Long Beach, California', None, None)
('ARSLMAU1187B98DA69', 'Recoil', '', None, None)
Total time to run: 2.739 seconds


### Time Table

In [12]:
start_time = time.time()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Drop table if it exists
time_table_drop = "DROP TABLE IF EXISTS public.time;"

# Create table
time_table_create = ("""
CREATE TABLE public.time (
    start_time TIMESTAMP PRIMARY KEY,
    hour INTEGER, 
    day INTEGER, 
    week INTEGER, 
    month INTEGER, 
    year INTEGER, 
    weekday INTEGER)
    DISTSTYLE AUTO;
""")

# copy data from staging table
time_table_insert = ("""
INSERT INTO public.time (start_time, hour, day, week, month, year, weekday) 
Select DISTINCT 
    start_time,
    EXTRACT(HOUR FROM start_time) As hour,
    EXTRACT(DAY FROM start_time) As day,
    EXTRACT(WEEK FROM start_time) As week,
    EXTRACT(MONTH FROM start_time) As month,
    EXTRACT(YEAR FROM start_time) As year,
    EXTRACT(DOW FROM start_time) As weekday
FROM (SELECT DISTINCT ts,'1970-01-01'::date + ts/1000 * interval '1 second' AS start_time FROM public.staging_events);
""")

# Execute queries
cursor.execute(time_table_drop)
conn.commit()
print("Dropped time table if it exists")

cursor.execute(time_table_create)
conn.commit()
print("Created time table")

cursor.execute(time_table_insert)
print("Inserted data from staging tables to time table")

# Print some data from table
query = ("""select * from public.time limit 10;""")
cursor.execute(query)
conn.commit()

print("Printing first 10 rows of time table")
i = 0
for row in cursor:
    i = i + 1
    print(row)
    if i == 10:
        break
    
print("Total time to run: {} seconds".format(round((time.time() - start_time),4)))

Dropped time table if it exists
Created time table
Inserted data from staging tables to time table
Printing first 10 rows of time table
(datetime.datetime(2018, 11, 2, 1, 34, 17), 1, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 9, 4, 16), 9, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 9, 26, 49), 9, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 9, 57, 56), 9, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 10, 21, 25), 10, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 10, 53, 11), 10, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 11, 20, 37), 11, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 11, 30, 25), 11, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 11, 44, 8), 11, 2, 44, 11, 2018, 5)
(datetime.datetime(2018, 11, 2, 12, 0, 45), 12, 2, 44, 11, 2018, 5)
Total time to run: 2.7466 seconds
